In [1]:
import os
import sys
import re
import pandas as pd
import json
from collections import Counter

In [2]:
news = open("news_channel_list.json", 'r', encoding="utf-8")
df = pd.read_json(news)
channels, urls = [], []
for channel, url in df["channels"]:
    channels.append(channel.replace(" ",""))
    urls.append(url)
    
print(len(channels))
len(set(urls))


115


115

In [3]:
pattern = r"(?<=output_).[^_]*(?=_)"
dir_name = "outputs_news"
output_file_name = "news_videos_Sep192023.csv"
channel_list = set()
files = [f"{dir_name}/{f}" for f in os.listdir(f"{dir_name}/") if f.endswith(".csv")]

for f in files:
    channel_name = re.search(pattern, f)[0]
    channel_list.add(channel_name)
    print(f)


outputs_news/output_1010WINS_0.csv
outputs_news/output_1010WINS_1.csv
outputs_news/output_1010WINS_2.csv
outputs_news/output_9NEWS_0.csv
outputs_news/output_9NEWS_1.csv
outputs_news/output_9NEWS_2.csv
outputs_news/output_ABC15Arizona_0.csv
outputs_news/output_ABC15Arizona_1.csv
outputs_news/output_ABC15Arizona_2.csv
outputs_news/output_ABC7Chicago_0.csv
outputs_news/output_ABC7Chicago_1.csv
outputs_news/output_ABC7Chicago_2.csv
outputs_news/output_ABC7LA_0.csv
outputs_news/output_ABC7LA_1.csv
outputs_news/output_ABC7LA_2.csv
outputs_news/output_ABC7NY_0.csv
outputs_news/output_ABC7NY_1.csv
outputs_news/output_ABC7NY_2.csv
outputs_news/output_ArsTechnica_0.csv
outputs_news/output_ArsTechnica_1.csv
outputs_news/output_ArsTechnica_2.csv
outputs_news/output_AssociatedPress_0.csv
outputs_news/output_AssociatedPress_1.csv
outputs_news/output_AssociatedPress_2.csv
outputs_news/output_AtlasObscura_0.csv
outputs_news/output_AtlasObscura_1.csv
outputs_news/output_AtlasObscura_2.csv
outputs_news/

In [4]:
print(len(set(channel_list)))
channel_list = list(channel_list)
channel_list[0]

115


'MilitaryTimes'

In [5]:
all_df_list = []
for channel in channel_list:
    channel_files = [f for f in files if f.startswith(f"{dir_name}/output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title
0,cMUSuMbyJ4s,MilitaryTimes,Watch Oklahoma airman take police on a 183-MPH...
1,6toXHd0xGdg,MilitaryTimes,Vietnam helicopter pilots describe the war fro...
2,8kk-YFywjpI,MilitaryTimes,Here's how to become a B-2 Spirt stealth bombe...
3,zZWA4KzBcn4,MilitaryTimes,"Becoming a Sailor, Part 1: Arrival Night at Bo..."
4,-Yk8NerPLhI,MilitaryTimes,"Becoming a Sailor, Part 2: Getting in the Water"
...,...,...,...
15569,ViCmxjDDLIM,ConsumerReports,4K Review: 2018 Honda Odyssey Quick Drive | Co...
15570,kCOgCwZzDBw,ConsumerReports,Why Not to Buy a Portable Air Conditioner | Co...
15571,RMWjK4cG-Yc,ConsumerReports,2014 Kia Forte first drive | Consumer Reports
15572,DryujWcJgjM,ConsumerReports,Kia Sportage SX review | Consumer Reports


In [6]:
Counter(df["channel_name"])

Counter({'9NEWS': 267,
         'WashingtonTopNews': 248,
         'NEWS9OklahomaCity': 246,
         'WKMGNews6ClickOrlando': 232,
         'HowStuffWorks': 231,
         'ABC7LA': 229,
         'MassLive': 225,
         'ConsumerReports': 225,
         'KARE11': 217,
         'AssociatedPress': 210,
         'FOX6NewsMilwaukee': 210,
         'IndyStar': 209,
         'WestVirginiaPublicBroadcasting': 209,
         'WLWT-Cincinnati': 202,
         'WHIO': 198,
         'HowToGeek': 194,
         'WBNS10TV': 150,
         'C-SPAN': 150,
         'ABC15Arizona': 149,
         'KCRA3': 149,
         'ForeignPolicy': 146,
         'FOX4Dallas-FortWorth': 144,
         'HawaiiNewsNow': 144,
         'FOX5NewYork': 143,
         'FOX5WashingtonDC': 143,
         'Fox32Chicago': 142,
         'KETV': 142,
         'ABC7NY': 141,
         'MadisonNews': 139,
         'NewsNation': 139,
         'AudacyMusic': 138,
         'KHOU11Houston': 138,
         'ThePost-Standard': 137,
         'Ben

In [7]:
df.to_csv(output_file_name, index=False, encoding="utf-8")

In [2]:
a = [1, 2]
b = [3, 4]

for c,d in a,b:
    print(d)

2
4
